In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb


## Preprocessing

### Drivers table

In [2]:
drivers_data_df = pd.read_csv("data/Crash_Reporting_-_Drivers_Data_20241116.csv", low_memory=False)


drivers_data_df["Collision Type"] = drivers_data_df["Collision Type"].str.upper()
drivers_data_df["Driver Distracted By"] = drivers_data_df["Driver Distracted By"].str.upper()
drivers_data_df["Vehicle Body Type"] = drivers_data_df["Vehicle Body Type"].str.upper()
drivers_data_df["Injury Severity"] = drivers_data_df["Injury Severity"].str.upper()

drivers_data_df["Light"] = drivers_data_df["Light"].str.upper()
drivers_data_df["Light"] = drivers_data_df["Light"].astype(str).str.replace(r'[-–—]+', '', regex=True).str.strip()
drivers_data_df["Light"] = drivers_data_df["Light"].str.replace(r'\s{2,}', ' ', regex=True)


drivers_data_df["Driver Substance Abuse"] = drivers_data_df["Driver Substance Abuse"].str.upper()
drivers_data_df["Driver Substance Abuse"] = drivers_data_df["Driver Substance Abuse"].apply(
    lambda desc: list(map(str.strip, desc.split(','))) if isinstance(desc, str) else desc
)

drivers_data_df["Non-Motorist Substance Abuse"] = drivers_data_df["Non-Motorist Substance Abuse"].str.upper()
drivers_data_df["Non-Motorist Substance Abuse"] = drivers_data_df["Non-Motorist Substance Abuse"].apply(
    lambda desc: list(map(str.strip, desc.split(','))) if isinstance(desc, str) else desc
)



drivers_data_df["Parked Vehicle"] = drivers_data_df["Parked Vehicle"].replace({"Yes" : True,
                                                                              "No" : False})




#Grouping of data
vehicle_body_type_mapping = {
    "Passenger Vehicles": [
        "PASSENGER CAR",
        "(SPORT) UTILITY VEHICLE",
        "SPORT UTILITY VEHICLE",
        "STATION WAGON",
        "AUTOCYCLE",
        "LOW SPEED VEHICLE",
        "RECREATIONAL VEHICLE"
    ],
    "Light Trucks / Vans": [
        "PICKUP TRUCK",
        "PICKUP",
        "OTHER LIGHT TRUCKS (10,000LBS (4,536KG) OR LESS)",
        "VAN",
        "VAN - CARGO",
        "CARGO VAN/LIGHT TRUCK 2 AXLES (OVER 10,000LBS (4,536 KG))",
        "VAN - PASSENGER (&LT;9 SEATS)",
        "VAN - PASSENGER (9 OR 12 SEATS)"
    ],
    "Heavy/Commercial Vehicles": [
        "MEDIUM/HEAVY TRUCKS 3 AXLES (OVER 10,000LBS (4,536KG))",
        "TRUCK TRACTOR",
        "SINGLE-UNIT TRUCK",
        "OTHER TRUCKS",
        "CONSTRUCTION EQUIPMENT (BACKHOE, BULLDOZER, ETC.)",
        "FARM VEHICLE",
        "FARM EQUIPMENT (TRACTOR, COMBINE HARVESTER, ETC.)"
    ],
    "Emergency / Law Enforcement Vehicles": [
        "POLICE VEHICLE/NON EMERGENCY",
        "POLICE VEHICLE/EMERGENCY",
        "AMBULANCE/EMERGENCY",
        "AMBULANCE/NON EMERGENCY",
        "FIRE VEHICLE/EMERGENCY",
        "FIRE VEHICLE/NON EMERGENCY"
    ],
    "Buses": [
        "TRANSIT BUS",
        "SCHOOL BUS",
        "BUS - TRANSIT",
        "BUS - SCHOOL",
        "BUS - MINI",
        "BUS - CROSS COUNTRY",
        "CROSS COUNTRY BUS",
        "BUS - OTHER TYPE",
        "OTHER BUS"
    ],
    "Motorcycles / Recreational / Specialty Vehicles": [
        "MOTORCYCLE",
        "MOTORCYCLE - 2 WHEELED",
        "MOTORCYCLE - 3 WHEELED",
        "MOPED",
        "MOPED OR MOTORIZED BICYCLE",
        "ALL TERRAIN VEHICLE (ATV)",
        "ALL-TERRAIN VEHICLE/ALL-TERRAIN CYCLE (ATV/ATC)",
        "SNOWMOBILE",
        "RECREATIONAL OFF-HIGHWAY VEHICLES (ROV)"
    ]
}

vehicle_group_mapping = {vehicle: group for group, vehicles in vehicle_body_type_mapping.items() for vehicle in vehicles}

drivers_data_df["Vehicle Type Group"] = drivers_data_df["Vehicle Body Type"].map(vehicle_group_mapping)

drivers_data_df["Crash Date/Time"] = pd.to_datetime(drivers_data_df["Crash Date/Time"],
                                                    format="%m/%d/%Y %I:%M:%S %p", errors="coerce")

drivers_data_df['Local Case Number'] = pd.to_numeric(drivers_data_df['Local Case Number'], errors='coerce')
drivers_data_df['Speed Limit'] = pd.to_numeric(drivers_data_df['Speed Limit'], errors='coerce').astype('Int64')
drivers_data_df['Vehicle Year'] = pd.to_numeric(drivers_data_df['Vehicle Year'], errors='coerce').astype('Int64')


#modify columns names for simpler querying
drivers_data_df.columns = [col.lower().replace(" ", "_") for col in drivers_data_df.columns]
drivers_data_df = drivers_data_df.rename(columns={"crash_date/time" : "crash_datetime"})

In [3]:
drivers_data_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188090 entries, 0 to 188089
Data columns (total 40 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   report_number                  188090 non-null  object        
 1   local_case_number              188070 non-null  float64       
 2   agency_name                    188090 non-null  object        
 3   acrs_report_type               188090 non-null  object        
 4   crash_datetime                 188090 non-null  datetime64[ns]
 5   route_type                     169696 non-null  object        
 6   road_name                      168741 non-null  object        
 7   cross-street_name              162133 non-null  object        
 8   off-road_description           17356 non-null   object        
 9   municipality                   19126 non-null   object        
 10  related_non-motorist           6040 non-null    object        
 11  

### Incidents table

In [4]:
incidents_df = pd.read_csv("data/Crash_Reporting_-_Incidents_Data_20241116.csv", low_memory=False)


In [5]:
incidents_redundant_columns = [
 'Local Case Number',
 'Agency Name',
 'ACRS Report Type',
 'Route Type',
 'Road Name',
 'Cross-Street Name',
 'Off-Road Description',
 'Municipality',
 'Related Non-Motorist',
 'Collision Type',
 'Weather',
 'Surface Condition',
 'Light',
 'Traffic Control',
 'Driver Substance Abuse',
 'Non-Motorist Substance Abuse',
 'Latitude',
 'Longitude',
 'Location']


incidents_df = incidents_df.drop(incidents_redundant_columns, axis=1)

In [6]:
incidents_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106601 entries, 0 to 106600
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Report Number         106601 non-null  object 
 1   Crash Date/Time       106601 non-null  object 
 2   Hit/Run               104906 non-null  object 
 3   Lane Direction        93146 non-null   object 
 4   Lane Type             17547 non-null   object 
 5   Number of Lanes       94261 non-null   object 
 6   Direction             93158 non-null   object 
 7   Distance              95021 non-null   float64
 8   Distance Unit         94273 non-null   object 
 9   Road Grade            92530 non-null   object 
 10  At Fault              106601 non-null  object 
 11  First Harmful Event   105951 non-null  object 
 12  Second Harmful Event  27871 non-null   object 
 13  Junction              79122 non-null   object 
 14  Intersection Type     50974 non-null   object 
 15  

In [7]:
incidents_df["Intersection Type"] = incidents_df["Intersection Type"].str.upper()
incidents_df["Direction"] = incidents_df["Direction"].str.upper()
incidents_df["Road Condition"] = incidents_df["Road Condition"].str.upper()
incidents_df["Lane Direction"] = incidents_df["Lane Direction"].apply(
    lambda desc: list(map(str.strip, desc.split(','))) if isinstance(desc, str) else desc
)



def convert_to_meters(distance, unit):
    
    if unit == "FEET":
        return distance * 0.3048  # Convert feet to meters
    elif unit == "MILE":
        return distance * 1609.34  # Convert miles to meters
    elif unit == "UNKNOWN":
        return None  # or np.nan if using NumPy
    else:
        return None  # Handle unexpected cases

# Example usage with sample data
# Assuming incidents_df is already loaded and contains columns "Distance" and "Distance Unit"
incidents_df["Distance meters"] = incidents_df.apply(lambda row: convert_to_meters(row["Distance"], row["Distance Unit"]), axis=1)
incidents_df = incidents_df.drop("Distance Unit", axis=1)



incidents_df = incidents_df.rename(columns={"Crash Date/Time" : "crash_datetime",
                                           "Hit/Run" : "hit"})

incidents_df.columns = [col.lower().replace(" ", "_") for col in incidents_df.columns]


In [8]:
incidents_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106601 entries, 0 to 106600
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   report_number         106601 non-null  object 
 1   crash_datetime        106601 non-null  object 
 2   hit                   104906 non-null  object 
 3   lane_direction        93146 non-null   object 
 4   lane_type             17547 non-null   object 
 5   number_of_lanes       94261 non-null   object 
 6   direction             93158 non-null   object 
 7   distance              95021 non-null   float64
 8   road_grade            92530 non-null   object 
 9   at_fault              106601 non-null  object 
 10  first_harmful_event   105951 non-null  object 
 11  second_harmful_event  27871 non-null   object 
 12  junction              79122 non-null   object 
 13  intersection_type     50974 non-null   object 
 14  road_alignment        92748 non-null   object 
 15  

### Non-motorist table

In [9]:
non_motorist_df = pd.read_csv("data/Crash_Reporting_-_Non-Motorists_Data_20241116.csv")


non_motorist_redundant_features = [
 'Local Case Number',
 'Agency Name',
 'ACRS Report Type',
 'Route Type',
 'Road Name',
 'Cross-Street Name',
 'Off-Road Description',
 'Municipality',
 'Related Non-Motorist',
 'Collision Type',
 'Weather',
 'Surface Condition',
 'Light',
 'Traffic Control',
 'Driver Substance Abuse',
 'Non-Motorist Substance Abuse',
 'Latitude',
 'Longitude',
 'Location']


non_motorist_df = non_motorist_df.drop(non_motorist_redundant_features, axis=1)


non_motorist_df["Pedestrian Actions"]  = non_motorist_df["Pedestrian Actions"].str.upper()
non_motorist_df["Safety Equipment"]  = non_motorist_df["Safety Equipment"].str.upper()
non_motorist_df["Injury Severity"]  = non_motorist_df["Injury Severity"].str.upper()




def categorize_pedestrian_type(pedestrian_type: str) -> str:
    
    pedestrian_type = pedestrian_type.lower()
    
    if "pedestrian" in pedestrian_type:
        return "PEDESTRIAN"
    elif "cyclist" in pedestrian_type or "bicyclist" in pedestrian_type:
        return "CYCLIST"
    elif "scooter" in pedestrian_type:
        return "SCOOTER"
    elif "unknown" in pedestrian_type:
        return "UNKNOWN"
    else:
        return "OTHER"

non_motorist_df["pedestrian_category"] = non_motorist_df["Pedestrian Type"].apply(categorize_pedestrian_type)


def categorize_safety_equipment(equipment: str) -> dict:


    if not isinstance(equipment, str):
        return {
            "is_helmet_present": np.nan,
            "is_lighting_present": np.nan,
            "is_reflective_wear": np.nan
        }
    
    equipment = equipment.lower()
    
    is_helmet_present = int("helmet" in equipment)
    is_lighting_present = int("lighting" in equipment)
    is_reflective_wear = int("reflective" in equipment or "reflectors" in equipment)
    
    return {
        "is_helmet_present": is_helmet_present,
        "is_lighting_present": is_lighting_present,
        "is_reflective_wear": is_reflective_wear
    }



non_motorist_df[['is_helmet_present', 'is_lighting_present', 'is_reflective_wear']] = non_motorist_df["Safety Equipment"].apply(lambda x: pd.Series(categorize_safety_equipment(x)))



non_motorist_df.columns = [col.lower().replace(" ", "_") for col in non_motorist_df.columns]



In [10]:
non_motorist_df.tail()

,report_number,crash_date/time,person_id,pedestrian_type,pedestrian_movement,pedestrian_actions,pedestrian_location,at_fault,injury_severity,safety_equipment,pedestrian_category,is_helmet_present,is_lighting_present,is_reflective_wear
6245,MCP3386001V,11/05/2024 05:02:00 PM,49D5A8E2-10F3-49DF-9830-998644D6407C,Pedestrian,Cross/Enter at Intersection,NONE (NO IMPROPER ACTION),Intersection Marked Crosswalk,No,SUSPECTED SERIOUS INJURY,NaN,PEDESTRIAN,NaN,NaN,NaN
6246,MCP23610035,11/12/2024 04:26:00 PM,FF6D68F8-84CA-41DA-ADDC-46F4829794E2,Pedestrian,Cross/Enter Not at Intersection,FAILURE TO YIELD RIGHT-OF-WAY,Travel Lane Other Location,Yes,POSSIBLE INJURY,NaN,PEDESTRIAN,NaN,NaN,NaN
6247,MCP3263004T,11/06/2024 08:57:00 AM,63A9432E-9754-4918-9F18-025871F0B5A9,Pedestrian,Cross/Enter at Intersection,NONE (NO IMPROPER ACTION),Intersection Marked Crosswalk,No,SUSPECTED MINOR INJURY,NaN,PEDESTRIAN,NaN,NaN,NaN
6248,MCP32330053,10/16/2024 05:56:00 PM,9B7A8E68-5EE6-4587-9281-68BA0C289C2B,Pedestrian,Cross/Enter Not at Intersection,UNKNOWN,Intersection Unmarked Crosswalk,No,POSSIBLE INJURY,NaN,PEDESTRIAN,NaN,NaN,NaN
6249,MCP3259005B,11/01/2024 03:15:00 PM,B127772C-F84B-4D67-BDD2-E6B8A1911719,Cyclist (non-electric),Walking/Cycling Along Roadway With Traffic (In...,"INATTENTIVE (TALKING, EATING, ETC.)",Shoulder/Roadside,No,SUSPECTED MINOR INJURY,HELMET,CYCLIST,1.0,0.0,0.0


## Load Data to DuckDB

### Initialize DuckDB

In [11]:
DB_PATH = "data/data.duckdb"

conn = duckdb.connect(database=DB_PATH, read_only=False)


In [12]:
conn.register("drivers_df", drivers_data_df)
conn.register("incidents_df", incidents_df)
conn.register("non_motorist_df", non_motorist_df)

# Create tables and persist the data
conn.execute("CREATE TABLE IF NOT EXISTS drivers AS SELECT * FROM drivers_df")
conn.execute("CREATE TABLE IF NOT EXISTS incidents AS SELECT * FROM incidents_df")
conn.execute("CREATE TABLE IF NOT EXISTS non_motorist AS SELECT * FROM non_motorist_df")


conn.sql("""UPDATE incidents
SET crash_datetime = strptime(crash_datetime, '%m/%d/%Y %I:%M:%S %p');
""")

In [13]:
conn.execute("SHOW TABLES;").fetchdf()

,name
0,drivers
1,drivers_df
2,incidents
3,incidents_df
4,non_motorist
5,non_motorist_df


In [14]:
conn.close()